In [3]:
%pip install watchdog


     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/79.1 kB ? eta -:--:--
   ---------------------------------------- 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [12]:
import os
import json
import time
import logging
import sys
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Configuration
STRUCTURE_FILE = "directory_structure.json"
BASE_DIRECTORY = "Medical-Project"

# Files and folders to ignore
IGNORE_FILES = {"node_modules", "public", ".git", ".next", "package.json", "package-lock.json", "yarn.lock", "dist", "build", "venv"}

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("directory_tool.log"), logging.StreamHandler()],
)

# Load existing directory structure
def load_existing_structure():
    if os.path.exists(STRUCTURE_FILE):
        with open(STRUCTURE_FILE, "r") as f:
            return json.load(f)
    return {}

# Save directory structure
def save_structure(structure):
    with open(STRUCTURE_FILE, "w") as f:
        json.dump(structure, f, indent=4)
    logging.info("✅ Updated directory structure saved.")

# Function to create project structure from JSON
def create_project_structure(base_path, structure):
    for key, value in structure.items():
        if key in IGNORE_FILES:
            continue
        
        current_path = os.path.join(base_path, key)
        os.makedirs(current_path, exist_ok=True)
        logging.info(f"📁 Created directory: {current_path}")

        if isinstance(value, dict):
            create_project_structure(current_path, value)
        elif isinstance(value, list):  
            for file in value:
                if file in IGNORE_FILES:
                    continue
                file_path = os.path.join(current_path, file)
                if not os.path.exists(file_path):
                    open(file_path, "w").close()
                    logging.info(f"📄 Created file: {file_path}")

# Extract an existing directory structure into JSON
def extract_directory_structure(root_dir):
    structure = {}
    for root, dirs, files in os.walk(root_dir):
        relative_path = os.path.relpath(root, root_dir)
        relative_path = "" if relative_path == "." else relative_path.replace("\\", "/")
        
        keys = relative_path.split("/") if relative_path else []
        current_level = structure
        for key in keys:
            current_level = current_level.setdefault(key, {})
        
        current_level[""] = [file for file in files if file not in IGNORE_FILES]
    
    save_structure(structure)
    logging.info(f"📌 Extracted directory structure from '{root_dir}'.")

# Monitor directory for changes
class DirectoryWatcher(FileSystemEventHandler):
    def on_created(self, event):
        if any(ignore in event.src_path for ignore in IGNORE_FILES):
            return
        
        if event.is_directory:
            dir_name = os.path.basename(event.src_path)
            directory = os.path.dirname(event.src_path).replace("\\", "/")
            relative_directory = os.path.relpath(directory, BASE_DIRECTORY)
            logging.info(f"📁 New directory detected: {dir_name} in {relative_directory}")
            update_json_structure(relative_directory, dir_name, is_directory=True)
        else:
            file_name = os.path.basename(event.src_path)
            directory = os.path.dirname(event.src_path).replace("\\", "/")
            relative_directory = os.path.relpath(directory, BASE_DIRECTORY)
            logging.info(f"📄 New file detected: {file_name} in {relative_directory}")
            update_json_structure(relative_directory, file_name, is_directory=False)

    def on_deleted(self, event):
        if any(ignore in event.src_path for ignore in IGNORE_FILES):
            return

        if event.is_directory:
            dir_name = os.path.basename(event.src_path)
            directory = os.path.dirname(event.src_path).replace("\\", "/")
            relative_directory = os.path.relpath(directory, BASE_DIRECTORY)
            logging.info(f"❌ Directory deleted: {dir_name} from {relative_directory}")
            remove_from_json_structure(relative_directory, dir_name, is_directory=True)
        else:
            file_name = os.path.basename(event.src_path)
            directory = os.path.dirname(event.src_path).replace("\\", "/")
            relative_directory = os.path.relpath(directory, BASE_DIRECTORY)
            logging.info(f"❌ File deleted: {file_name} from {relative_directory}")
            remove_from_json_structure(relative_directory, file_name, is_directory=False)

# Update JSON when files/directories are created
def update_json_structure(directory, name, is_directory):
    structure = load_existing_structure()
    keys = directory.split("/") if directory else []
    current_level = structure
    for key in keys:
        current_level = current_level.setdefault(key, {})

    if is_directory:
        current_level.setdefault(name, {})
    else:
        current_level.setdefault("", []).append(name)
    
    save_structure(structure)
    logging.info(f"🔄 Updated JSON structure with new {'directory' if is_directory else 'file'}: {name}")

# Update JSON when files/directories are removed
def remove_from_json_structure(directory, name, is_directory):
    structure = load_existing_structure()
    keys = directory.split("/") if directory else []
    current_level = structure
    for key in keys:
        current_level = current_level.get(key, {})

    if is_directory:
        current_level.pop(name, None)
    else:
        if "" in current_level and name in current_level[""]:
            current_level[""].remove(name)
    
    save_structure(structure)
    logging.info(f"🔄 Updated JSON structure by removing {'directory' if is_directory else 'file'}: {name}")

# Start monitoring directory
def start_directory_watcher():
    observer = Observer()
    event_handler = DirectoryWatcher()
    observer.schedule(event_handler, BASE_DIRECTORY, recursive=True)
    observer.start()
    logging.info(f"👀 Watching '{BASE_DIRECTORY}' for changes...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Main function
def main():
    print("Select an option:")
    print("1. Create project structure")
    print("2. Extract directory structure")
    print("3. Watch directory for changes")
    choice = input("Enter your choice (1/2/3): ").strip()

    if choice == "1":
        structure = load_existing_structure()
        create_project_structure(BASE_DIRECTORY, structure)
        logging.info("✅ Project structure created.")
    elif choice == "2":
        extract_directory_structure(BASE_DIRECTORY)
    elif choice == "3":
        start_directory_watcher()
    else:
        print("❌ Invalid choice! Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()


Select an option:
1. Create project structure
2. Extract directory structure
3. Watch directory for changes


2025-01-29 15:10:39,899 - INFO - ✅ Updated directory structure saved.
2025-01-29 15:10:39,899 - INFO - 📌 Extracted directory structure from 'Medical-Project'.


Create Project Structure

In [ ]:
% python directory_tool.py create

In [ ]:
% python directory_tool.py extract

In [ ]:
% python directory_tool.py watch